In [38]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib

pd.set_option("display.max_rows", None)
database_name = "TSX_Quality.sqlite"

### Utility function to get prices from our local database  

In [39]:
def get_prices_for(symbol):
    conn = sqlite3.connect(database_name)
    
    # Make sur all dates are in ascending order
    sql = f"SELECT * FROM Prices_Daily WHERE Ticker = '{symbol}' ORDER BY UPPER(Ticker) ASC, Date ASC"
    prices = pd.read_sql_query(sql, conn, index_col="Date")
    prices.index = pd.to_datetime(prices.index)
    
    # Clean up CSV data to make sure we have only floats and no "-" values
    prices.replace("-", np.NaN, inplace=True)
    #prices["Volume"].replace(0, np.NaN, inplace=True)
    prices["Open"] = prices["Open"].astype(float)
    prices["High"] = prices["High"].astype(float)
    prices["Low"]  = prices["Low"].astype(float)
    prices["Close"]= prices["Close"].astype(float)
    
    # Required for finTA
    prices.rename(columns={"Open":"open", "High":"high", "Low":"low", "Close":"close", "Volume":"volume"}, inplace=True)
    prices.index.rename("date", inplace=True) 
    
    return prices

### Indicator calculations added to the prices dataframe

In [60]:
symbol = "LI"
prices_df = get_prices_for(symbol)
prices_df.tail(10)

,Ticker,open,high,low,close,volume
date,,,,,,
2022-01-11,LI,3.98,4.18,3.91,4.05,618700.0
2022-01-12,LI,4.10,4.18,4.03,4.03,322700.0
2022-01-13,LI,3.98,3.98,3.78,3.86,417200.0
2022-01-14,LI,3.68,3.74,3.62,3.73,617800.0
2022-01-17,LI,3.62,3.62,3.49,3.54,333100.0
2022-01-18,LI,3.38,3.39,3.26,3.28,958900.0
2022-01-19,LI,3.16,3.18,2.97,3.09,1429200.0
2022-01-20,LI,3.12,3.59,3.12,3.24,1039700.0
2022-01-21,LI,3.14,3.19,2.98,3.12,840600.0


In [61]:
# Let's add all our values to calculate teh ATR indicators 
prices_df["prev_close"]        = prices_df["close"].shift()
prices_df["high-low"]          = prices_df["high"] - prices_df["low"]
prices_df["high-prev_close"]   = np.abs(prices_df["high"] - prices_df["prev_close"])
prices_df["low-prev_close"]    = np.abs(prices_df["low"] - prices_df["prev_close"])
prices_df["true_range"]        = prices_df[["high-low", "high-prev_close", "low-prev_close"]].max(axis=1)
    
prices_df["ATR10"] = prices_df["true_range"].rolling(window=10).mean()

# Supertrend indicator
# basic_upper_band = ((high + low) / 2) + (multiplier * atr)
# basic_lower_band = ((high + low) / 2) - (multiplier * atr)
atr_multiplier = 3
prices_df["basic_upper_band"] = (prices_df["high"] + prices_df["low"]) / 2 + (atr_multiplier * prices_df["ATR10"])
prices_df["upper_band"]       = prices_df["basic_upper_band"]
prices_df["basic_lower_band"] = (prices_df["high"] + prices_df["low"]) / 2 - (atr_multiplier * prices_df["ATR10"])
prices_df["lower_band"]       =  prices_df["basic_lower_band"]


prices_df.tail(10)


,Ticker,open,high,low,close,volume,prev_close,high-low,high-prev_close,low-prev_close,true_range,ATR10,basic_upper_band,upper_band,basic_lower_band,lower_band
date,,,,,,,,,,,,,,,,
2022-01-11,LI,3.98,4.18,3.91,4.05,618700.0,3.90,0.27,0.28,0.01,0.28,0.268,4.849,4.849,3.241,3.241
2022-01-12,LI,4.10,4.18,4.03,4.03,322700.0,4.05,0.15,0.13,0.02,0.15,0.264,4.897,4.897,3.313,3.313
2022-01-13,LI,3.98,3.98,3.78,3.86,417200.0,4.03,0.20,0.05,0.25,0.25,0.265,4.675,4.675,3.085,3.085
2022-01-14,LI,3.68,3.74,3.62,3.73,617800.0,3.86,0.12,0.12,0.24,0.24,0.275,4.505,4.505,2.855,2.855
2022-01-17,LI,3.62,3.62,3.49,3.54,333100.0,3.73,0.13,0.11,0.24,0.24,0.284,4.407,4.407,2.703,2.703
2022-01-18,LI,3.38,3.39,3.26,3.28,958900.0,3.54,0.13,0.15,0.28,0.28,0.280,4.165,4.165,2.485,2.485
2022-01-19,LI,3.16,3.18,2.97,3.09,1429200.0,3.28,0.21,0.10,0.31,0.31,0.261,3.858,3.858,2.292,2.292
2022-01-20,LI,3.12,3.59,3.12,3.24,1039700.0,3.09,0.47,0.50,0.03,0.50,0.283,4.204,4.204,2.506,2.506
2022-01-21,LI,3.14,3.19,2.98,3.12,840600.0,3.24,0.21,0.05,0.26,0.26,0.294,3.967,3.967,2.203,2.203
